# 데이터 입력받기

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
print(data.shape)
data.head()

In [ ]:
del data['ID_code']

# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(data)

# PCA(차원축소)

In [ ]:
y = data.pop('target')
X = data

In [ ]:
#train / test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.decomposition import PCA

# test set을 transform 할때는 train의 fit으로 !
pca = PCA(n_components=10) # n_components는 feature의 개수이다.

pca.fit(X_train)

X_train_pca = pd.DataFrame(pca.transform(X_train))
X_test_pca = pd.DataFrame(pca.transform(X_test))

# SGDClassfier -전체 데이터를 작은 데이터셋으로 나눠서 학습하고 점점 업데이트를 시키는 방법
### loss="hinge"로 두면 svm을 적용할 수 있다.

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf.fit(X_train_pca,y_train)

In [ ]:
clf.score(X_test_pca, y_test) # scaler적용후 pca를 10으로 하고 기본 sgdclassfier를 쓸경우 accuracy가 0.82정도 나온다!

# Grid Search를 통한 적절한 하이퍼파라미터 찾기

In [ ]:
%%time

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# pipe = Pipeline([
#     ('scl', StandardScaler()), ('sgd', SGDClassifier())])

params = {
    "max_iter" : [10, 50, 100, 500, 1000],
    "n_iter" : [1, 5, 10],
    "alpha" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    "penalty": ["none", "l1", "l2"],
}

grid = GridSearchCV(estimator=SGDClassifier(), param_grid=params, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_pca, y_train)

print(grid.best_params_)
print(grid.best_score_)

In [ ]:
clf = SGDClassifier(alpha=1, n_iter=1, penalty="l2", max_iter=500)
clf.fit(X_train_pca, y_train)

In [ ]:
clf.score(X_test_pca, y_test) # sgd class fier만 

In [ ]:
%%time

best_params_with_PCA = dict()

for i in range(4, 14):
    pca = PCA(n_components=i) # n_components는 feature의 개수이다.

    pca.fit(X_train)

    X_train_pca = pd.DataFrame(pca.transform(X_train))
    X_test_pca = pd.DataFrame(pca.transform(X_test))
    
    params = {
        "max_iter" : [10, 50, 100, 500, 1000],
        "n_iter" : [1, 5, 10],
        "alpha" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        "penalty": ["none", "l1", "l2"],
    }

    grid = GridSearchCV(estimator=SGDClassifier(), param_grid=params, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train_pca, y_train)

    best_params_with_PCA[i] = (grid.best_params_, grid.best_score_)

In [ ]:
best_params_with_PCA

In [ ]:
'''
PCA의 차이는 그다지 유의미해보이지 않는다.
10개의 feature말고 7개로 예측을 해보겠다.
'''
pca = PCA(n_components=7) # n_components는 feature의 개수이다.

pca.fit(X_train)

X_train_pca = pd.DataFrame(pca.transform(X_train))
X_test_pca = pd.DataFrame(pca.transform(X_test))

clf = SGDClassifier(alpha=1, max_iter=10, n_iter=10, penalty="l2")
clf.fit(X_train_pca, y_train)
clf.score(X_test_pca, y_test) # 10개로 차원축소한 score와 비슷하다.